# Machine Learning Engineer Nanodegree

# Capstone: Predicting Loan Defaults In Peer-To-Peer Lending

## I. Getting Started
In this project, we will analyze a dataset containing data on potential borrowers. The goal of this project is to build a model predicting the loan default of potential borrowers. 

The dataset for this project can be found on [Lending Club](https://www.lendingclub.com/info/download-data.action).

Run the code block below to load the wholesale customers dataset, along with a few of the necessary Python libraries required for this project. You will know the dataset loaded successfully if the size of the dataset is reported.


In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display # Allows the use of display() for DataFrames

# Pretty display for notebooks
%matplotlib inline
plt.style.use('fivethirtyeight')

# Load the accepted loan dataset 
# low_memory and skiprows in read_csv because the file is big
try:
    loan_data = pd.read_csv("LoanStats3a.csv", low_memory = False, skiprows = 1)
    print "The loan dataset has {} samples with {} features.".format(*loan_data.shape)
except:
    print "The loan dataset could not be loaded. Is the dataset missing?"

The loan dataset has 42538 samples with 111 features.


## II. Analysis

## Introduction To The Data

(explain the process of Lending club loan approval)

The dictionary data file is provided with the project in order to refer to it later in our data exploration. This contains information about the various columns and will be useful when we clean up the dataset. The data being used is the data from 2007 to 2011 mostly because when can be almost certain that all the loans have been either repaid or defaulted upon.

In [2]:
half_count = len(loan_data) / 2
loan_data = loan_data.dropna(thresh=half_count, axis=1)
loan_data = loan_data.drop(['desc', 'url'],axis=1)
loan_data.to_csv('loans_2007.csv', index=False)

In [3]:
loans_2007 = pd.read_csv('loans_2007.csv', low_memory = False)
loans_2007.drop_duplicates()

loans_2007.iloc[0]

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

In [4]:
loans_2007.shape[1]

52

The Dataframe is cumbersome and we had to set the `low_memory` to `False` to avoid a warning message from the notebook. This is due to the numerous columns of the dataset. Let us explore the dataset with the data dictionary this will be useful as we go through the data and try to clean it.

We will need to be careful about data from the future, this type of leakage could throw off the useful predictions of our model. A clean example is information about the borrower after the loan was approved, this is not data that we would have at our disposal. 

We will be splitting the columns in 4 giving us 13 features to analysis and try to make sense of. This part is crucial in order to understand the data and avoid error while fitting our machine learning model later on. 


### Build A Table To Analyse And Visualize
We will build a table with 2 csv files. We will use the first entry of the `loans_2007.csv` file to explore the meaning of the 52 columns. 

In [5]:
first_entry = loans_2007.iloc[0]
first_entry.to_csv('first_entry.csv', index = True)

In [6]:
description = pd.read_csv('LCDataDictionary.csv')

description.shape

(115, 2)

In [7]:
import csv
list_first_entry = open('first_entry.csv', 'r')
first_csvreader = csv.reader(list_first_entry)
first_list = list(first_csvreader)

list_data_dictio = open('LCDataDictionary.csv', 'r')
second_csvreader = csv.reader(list_data_dictio)
second_list = list(second_csvreader)

table = []
for row in first_list:
    table.append(row[0])

new_table = []
for col in second_list:
    if col[0] in table:
        new_table.append(col)

In [8]:
table

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'pub_rec_bankruptcies',
 'tax_liens']

In [9]:
new_table

[['acc_now_delinq',
  'The number of accounts on which the borrower is now delinquent.'],
 ['addr_state', 'The state provided by the borrower in the loan application'],
 ['annual_inc',
  'The self-reported annual income provided by the borrower during registration.'],
 ['application_type',
  'Indicates whether the loan is an individual application or a joint application with two co-borrowers'],
 ['chargeoff_within_12_mths', 'Number of charge-offs within 12 months'],
 ['collection_recovery_fee', 'post charge off collection fee'],
 ['collections_12_mths_ex_med',
  'Number of collections in 12 months excluding medical collections'],
 ['delinq_2yrs',
  "The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years"],
 ['delinq_amnt',
  'The past-due amount owed for the accounts on which the borrower is now delinquent.'],
 ['dti',
  'A ratio calculated using the borrower\xe2\x80\x99s total monthly debt payments on the total debt obligations, ex

### First Set Of Features
By analyzing the first 13 features, we understand that we can remove the *`id`* and *`member_id`* feature because it does not tell us anything about the person, these are arbitrary features given by Lending Club.

The *`funded_amnt`* and *`funded_amnt_inv`* are both features about the future the loan has been approved at that point and cannot be considered in our model. *`grade`* and *`sub_grade`* are recurring data  that we already have with (or a different angle on) the *`int_rate`* feature, thus we can dispose of them. 

Finally, although it could be an area of improvement in our model, the *`emp_title`* feature would be a hard feature to evaluate. Some form of sentiment analysis would need to be used and certain metrics would need to provide a good estimate of a title's meaning and value in the lending context. We will refrain ourselves in delving deeper, but we noticed this could be an interesting data science side project.  

In [10]:
#We drop the columns enumerated in the cell above.
loans_2007 = loans_2007.drop(['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 
                              'grade', 'sub_grade', 'emp_title'], axis =1)

### Second Set Of Features
The next 13 features contain *`issue_d`*, this is data about the month the loan was funded, this means it leaks a future information, we need to remove it. The *`zip_code`* is a column that does not add any value because we already have the state adress with *`addr_state`*. The *`zip_code`* could be used with other economic data to uncover a relationship with the environnement in which a person lives and the risk of default, but this could be seen as discriminatory practice and we only have the first 3 digits of the code area.

In [11]:
loans_2007 = loans_2007.drop(['issue_d','zip_code'], axis =1)

### Third Set Of Features
The following 5 variables are all about the future, they inform us about how the repayment is going, *`out_prncp`*, *`out_prncp_inv`*, *`total_pymnt`*, *`total_pymnt_inv`*, *`total_rec_prncp`*. We need to remove them from our model. The *`total_rec_int`* is about the interest received to date (meaning the loan has been approved) and *`total_rec_late_fee`* is about the interest that are late. These 2 variables need to be removed from the dataset.

In [12]:
loans_2007 = loans_2007.drop(['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
                              'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee'], axis =1)

### Fourth Set Of Features
Finally, the last set of 13 features contain *`recoveries`* and *`collection_recovery_fee`* which are both leak from the future about the loan. And *`last_pymnt_d`* and *`last_pmyny_amnt`* are about the end of repayment, which means we cannot know that in advance.    

In [13]:
loans_2007 = loans_2007.drop(['recoveries', 'collection_recovery_fee', 
                              'last_pymnt_d', 'last_pymnt_amnt'], axis =1)

In [14]:
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               D

We now have 32 columns to work with. We removed 20 columns of unuseful information and this will make the data easier to process and fit with the machine learning algorithm. 

### Target Column
The target column is a critical part of developping this type of machine learning algorithm because it tries to make prediction based on the outcome that we want. In this particular case, we want to predict the loan status (*`loan_status`*) which can take many values (9) in total. 

In [15]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             34085
Charged Off                                             5662
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Current                                                   19
Late (31-120 days)                                         9
In Grace Period                                            8
Late (16-30 days)                                          2
Default                                                    1
Name: loan_status, dtype: int64

We have 9 possibility for *`loan_status`* and only 2 values are important in our model's binary classification; fully paid and charged off. These 2 values indicate the result of the loan outcome. We will remove the other possibilities and avoid "translating" those values into the binary possibility (fully paid or charged off).

In [16]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

### Final Data Cleaning
Single value columns are not information that helps our model, it does not provide any insight into the likelihood of default or repayment, and will be removed from the dataset.

In [17]:
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis = 1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [18]:
loans_2007.shape

(39747, 23)

We were able to remove 9 columns and we are left with 23 features to work with. 

## III. Methodology

### Preparing The Features
We start with the filtered *loans_2007.csv* from the previous analysis. Do we have many Null values in the file loans_2007? We will look at how we can handle these values.  

In [19]:
null_counts = loans_2007.isnull().sum()
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


We have 3 columns with missing values: 10 with *`title`*, 50 with *`revol_util`* and 697 with *`pub_rec_bankruptcies`* . Instead of removing the columns as we did earlier, we will remove the rows. We consider rows as incomplete (in a real life setting, we would reject the application simply because it is not complete).

In [20]:
loans_2007 = loans_2007.drop("pub_rec_bankruptcies", axis=1)
loans_2007 = loans_2007.dropna(axis=0)

### Data Type
What about the types of columns?

In [22]:
print(loans_2007.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


We need to deal with the object data, because we cannot process it with scikit-learn, in order to use them in our model, we will need to convert them to numerical types. 

In [24]:
object_columns_df = loans_2007.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Dec-2016
Name: 0, dtype: object


### Text Data To Categorical Data
Most of the text data can be adapted to be processed by the scikit-learn algorithms. 



(insert explanation)

In [26]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans_2007[c].value_counts())

RENT        18860
MORTGAGE    17671
OWN          3055
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16887
Verified           12809
Source Verified     9989
Name: verification_status, dtype: int64
10+ years    8886
< 1 year     4573
2 years      4387
3 years      4090
4 years      3429
5 years      3279
1 year       3236
6 years      2224
7 years      1770
8 years      1480
9 years      1259
n/a          1072
Name: emp_length, dtype: int64
 36 months    29041
 60 months    10644
Name: term, dtype: int64
CA    7091
NY    3811
FL    2866
TX    2727
NJ    1845
IL    1523
PA    1514
VA    1406
GA    1398
MA    1342
OH    1221
MD    1052
AZ     877
WA     840
CO     790
NC     786
CT     753
MI     722
MO     685
MN     613
NV     497
SC     472
WI     458
AL     449
OR     448
LA     436
KY     327
OK     299
KS     271
UT     258
AR     245
DC     212
RI     199
NM     189
WV     177
HI     173
NH     172
DE     113
MT      85
WY      83
AK      

The reason the loan was asked for: 

In [29]:
print(loans_2007["purpose"].value_counts())

debt_consolidation    18636
credit_card            5129
other                  3982
home_improvement       2979
major_purchase         2181
small_business         1823
car                    1549
wedding                 947
medical                 693
moving                  581
house                   382
vacation                380
educational             320
renewable_energy        103
Name: purpose, dtype: int64


In [30]:
print(loans_2007["title"].value_counts())

Debt Consolidation                        2185
Debt Consolidation Loan                   1727
Personal Loan                              661
Consolidation                              514
debt consolidation                         507
Home Improvement                           357
Credit Card Consolidation                  357
Debt consolidation                         334
Small Business Loan                        328
Credit Card Loan                           318
Personal                                   308
Consolidation Loan                         256
Home Improvement Loan                      248
personal loan                              234
personal                                   220
Loan                                       213
Wedding Loan                               209
consolidation                              205
Car Loan                                   204
Other Loan                                 192
Credit Card Payoff                         155
Wedding      

We need to encode categorical value with dummy variables. 

In [31]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans_2007 = loans_2007.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans_2007["int_rate"] = loans_2007["int_rate"].str.rstrip("%").astype("float")
loans_2007["revol_util"] = loans_2007["revol_util"].str.rstrip("%").astype("float")
loans_2007 = loans_2007.replace(mapping_dict)

Now, let's encode the previous variable.

In [32]:
cat_columns = ["home_ownership", "verification_status", "emp_length", "purpose", "term"]
dummy_df = pd.get_dummies(loans_2007[cat_columns])
loans_2007 = pd.concat([loans_2007, dummy_df], axis=1)
loans_2007 = loans_2007.drop(cat_columns, axis=1)